In [1]:
#Analysis and data-loading functions
"""
Objective:
    *Docs
        - Create summary to easily access description for different data-attributes
    *Loading
        - Create functions that load arbitrairy amount of json-data for the same
        type of statistics.
        - Create functions to concatinate json-data of different type
    *Analysis
        - Visualize data-attributes in graphs
"""

'\nObjective:\n    *Docs\n        - Create summary to easily access description for different data-attributes\n    *Loading\n        - Create functions that load arbitrairy amount of json-data for the same\n        type of statistics.\n        - Create functions to concatinate json-data of different type\n    *Analysis\n        - Visualize data-attributes in graphs\n'

In [21]:
from directory import Directory
import pandas as pd
from pprint import pprint
dirs = Directory()

In [24]:


dirs = Directory()

def load_player_stats(start_year, end_year=None):
    """Load player_stats json files into a container
        Args:
            start_year(int): year of player_stats json
            end_year(int): year of player_stats json
    """
    if end_year == None:
        file = f'EN_PR_{start_year}_playerstats.json'
        stats_file = dirs.load_json(file, '..', 'json', 'params', 'stats')
        return stats_file
    else:
        all_files = [f'EN_PR_{year}_playerstats.json' for year in range(start_year,end_year+1)]
        container = []
        for file in all_files:
            stats_file = dirs.load_json(file, '..', 'json', 'params', 'stats')
            container.append(stats_file)
        return container
    
data = load_player_stats(2017)

In [39]:
for d in data:
    stats = d['stats']
    for i in stats:
        print(i)



        
        
        
#df = df = pd.DataFrame(d["stats"] for d in data)
#df.head()

{'id': 4985.0}
{'name': 'accurate_back_zone_pass', 'value': 651.0, 'description': 'Todo: accurate_back_zone_pass', 'additionalInfo': {}}
{'name': 'accurate_chipped_pass', 'value': 44.0, 'description': 'Todo: accurate_chipped_pass', 'additionalInfo': {}}
{'name': 'accurate_cross', 'value': 10.0, 'description': 'Todo: accurate_cross', 'additionalInfo': {}}
{'name': 'accurate_cross_nocorner', 'value': 10.0, 'description': 'Todo: accurate_cross_nocorner', 'additionalInfo': {}}
{'name': 'accurate_flick_on', 'value': 3.0, 'description': 'Todo: accurate_flick_on', 'additionalInfo': {}}
{'name': 'accurate_fwd_zone_pass', 'value': 922.0, 'description': 'Todo: accurate_fwd_zone_pass', 'additionalInfo': {}}
{'name': 'accurate_launches', 'value': 2.0, 'description': 'Todo: accurate_launches', 'additionalInfo': {}}
{'name': 'accurate_layoffs', 'value': 17.0, 'description': 'Todo: accurate_layoffs', 'additionalInfo': {}}
{'name': 'accurate_long_balls', 'value': 33.0, 'description': 'Todo: accurate_l

In [ ]:
def load_fixture_stats():
    pass

In [22]:
ops = {
  "+": (lambda a, b: a + b),
  "-": (lambda a, b: a - b),
  "*": (lambda a, b: a * b),
  "/": (lambda a, b: a / b)
}

def eval(expression):
    tokens = expression.split()
    stack = []

    for token in tokens:
        if token in ops:
            arg2 = stack.pop()
            arg1 = stack.pop()
            result = ops[token](arg1, arg2)
            stack.append(result)
        else:
            stack.append(int(token))

    return stack.pop()
      
print(eval("4 5 3 4 - + *"))


    

16
